# Show4DSTEM — MoS₂ abTEM Multislice

Simulate atomic-resolution 4D-STEM data of MoS₂ [001] zone axis using [abTEM](https://abtem.github.io/abtem/) multislice.
Square supercell ensures isotropic k-space sampling (circular BF disk, not oval).
Data is wrapped in a `Dataset4dstem` for automatic calibration extraction.

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %env ANYWIDGET_HMR=1
except Exception:
    pass  # autoreload unavailable (Colab Python 3.12+)

In [2]:
import abtem
import numpy as np
from ase.io import read
from quantem.core.datastructures import Dataset4dstem
from quantem.widget import Show4DSTEM

## Crystal structure

Read MoS₂ from CIF, orthogonalize the hexagonal cell (γ=120° → 90°), and create a
**square** supercell. A square real-space cell gives square k-space pixels, so the
BF disk is circular (not oval). Larger supercell = more scan positions = better
atomic-resolution image.

In [3]:
atoms = read("/Users/macbook/repos/quantem.widget/data/MoS2.cif")
atoms_ortho = abtem.orthogonalize_cell(atoms)
a, b = atoms_ortho.cell.lengths()[0], atoms_ortho.cell.lengths()[1]
print(f"Formula: {atoms.get_chemical_formula()}")
print(f"Orthogonal cell: {a:.4f} × {b:.4f} Å")

# Target: 128×128 scan at 0.5 Å step → need ~64 Å square supercell
target = 64.0
nx = max(1, round(target / a))
ny = max(1, round(target / b))
while abs(nx * a - ny * b) / max(nx * a, ny * b) > 0.02:
    if nx * a > ny * b:
        ny += 1
    else:
        nx += 1

slab = atoms_ortho.repeat((nx, ny, 1))
Lx, Ly = slab.cell.lengths()[0], slab.cell.lengths()[1]
print(f"Repeat: ({nx}, {ny}, 1) → {len(slab)} atoms")
print(f"Supercell: {Lx:.1f} × {Ly:.1f} Å (ratio: {Lx/Ly:.3f})")
print(f"Expected scan: ~{int(Lx/0.5)}×{int(Ly/0.5)} positions")

Formula: Mo2S4
Orthogonal cell: 3.1854 × 5.5173 Å
Repeat: (21, 12, 1) → 3024 atoms
Supercell: 66.9 × 66.2 Å (ratio: 1.010)
Expected scan: ~133×132 positions


## Multislice simulation

300 kV probe (21.4 mrad convergence), pixelated detector. Potential sampling of
0.08 Å is sufficient for this convergence angle (Nyquist criterion). The raw
detector grid from abTEM depends on the potential sampling — we bin it down to
64×64 for a clean final shape of **(128, 128, 64, 64)**.

Note: abTEM GPU acceleration requires NVIDIA CUDA (via CuPy). On macOS (Apple
Silicon), the simulation runs on CPU only.

In [ ]:
from scipy.ndimage import zoom as ndizoom
import time

# 0.08 Å sampling is sufficient for 21.4 mrad at 300 kV (Nyquist)
# Using 0.04 Å would give 1673×1656 grid — 4× slower with no visual benefit
potential = abtem.Potential(slab, sampling=0.08, slice_thickness=1.0)
probe = abtem.Probe(energy=300e3, semiangle_cutoff=21.4, sampling=potential.sampling)

max_angle = 100  # mrad
scan_sampling = 0.5  # Å
det_target = 64  # target detector pixels per side

scan = abtem.GridScan(start=(0, 0), end=(Lx, Ly), sampling=scan_sampling)
detector = abtem.PixelatedDetector(max_angle=max_angle)

print(f"Potential grid: {potential.gpts}")
print(f"Probe: 300 kV, α = 21.4 mrad")
print(f"Scan: {scan_sampling} Å step")
print(f"Running 4D-STEM scan...")

t0 = time.time()
measurement = probe.scan(potential, scan=scan, detectors=detector)
elapsed = time.time() - t0
print(f"Done in {elapsed:.1f}s")

raw = np.asarray(measurement.array, dtype=np.float32)
print(f"Raw shape: {raw.shape}")

# Crop scan to exactly 128×128, then bin detector to 64×64
scan_target = 128
sr = min(raw.shape[0], scan_target)
sc = min(raw.shape[1], scan_target)
data = raw[:sr, :sc]

# Bin detector: average neighboring pixels to reach 64×64
dr, dc = data.shape[2], data.shape[3]
if dr != det_target or dc != det_target:
    factor_r = det_target / dr
    factor_c = det_target / dc
    data = ndizoom(data, (1, 1, factor_r, factor_c), order=1).astype(np.float32)

print(f"Final shape: {data.shape}")
print(f"Memory: {data.nbytes / 1e6:.0f} MB")

Potential grid: (837, 828)
Probe: 300 kV, α = 21.4 mrad
Scan: 0.5 Å step
Running 4D-STEM scan...
Done in 0.0s


## Show4DSTEM via Dataset4dstem

Wrap the 4D array in a `Dataset4dstem` with calibration metadata. Show4DSTEM
auto-extracts `pixel_size` and `k_pixel_size` from the dataset — no manual
calibration needed.

In [ ]:
# k-space calibration: detector spans ±max_angle → 2*max_angle total
k_px = 2 * max_angle / data.shape[-1]  # mrad/px

ds = Dataset4dstem.from_array(
    array=data,
    name="MoS₂ [001] 4D-STEM",
    sampling=[scan_sampling, scan_sampling, k_px, k_px],
    units=["Å", "Å", "mrad", "mrad"],
)
print(f"Dataset: {ds.name}, shape: {ds.array.shape}")
print(f"  scan: {ds.sampling[0]} Å/px, detector: {k_px:.2f} mrad/px")

w = Show4DSTEM(ds)
w.auto_detect_center()
w.roi_circle()
print(f"BF disk: center=({w.center_row:.1f}, {w.center_col:.1f}), radius={w.bf_radius:.1f} px")
w